# Data Prepration

# Converting .coco file into yolo supported format

In [3]:
import json
import os

coco_file_path = 'DataSet/labels/train/_annotations.coco.json'
output_dir = 'YDataSet/labels/train/'

with open(coco_file_path) as f:
    data = json.load(f)

os.makedirs(output_dir, exist_ok=True)

categories = {cat['id']: cat['name'] for cat in data['categories']}

for image in data['images']:
    image_id = image['id']
    file_name = image['file_name']
    width = image['width']
    height = image['height']

    annotations = [ann for ann in data['annotations'] if ann['image_id'] == image_id]
    
    yolo_annotations = []
    for ann in annotations:
        category_id = ann['category_id']
        bbox = ann['bbox']

        x_min, y_min, bbox_width, bbox_height = bbox
        x_center = (x_min + bbox_width / 2) / width
        y_center = (y_min + bbox_height / 2) / height
        w = bbox_width / width
        h = bbox_height / height
        
        yolo_annotations.append(f"{category_id} {x_center} {y_center} {w} {h}")
    
    yolo_file_path = os.path.join(output_dir, os.path.splitext(file_name)[0] + '.txt')
    with open(yolo_file_path, 'w') as f:
        f.write("\n".join(yolo_annotations))

print("Conversion complete.")


Conversion complete.


# Train Object Detection Model

In [ ]:
from ultralytics import YOLO


model = YOLO("yolov8n.yaml")

results = model.train(data="config.yaml", epochs=20)

# Evaluating the model

In [ ]:
val_results = model.val(data='config.yaml')
val_results.print()

## Extract metrics from training results

### Plot Loss over epochs

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(metrics['train/box_loss'], label='Box Loss')
plt.plot(metrics['train/obj_loss'], label='Objectness Loss')
plt.plot(metrics['train/cls_loss'], label='Classification Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

### Plot Precision and Recall

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(metrics['metrics/precision'], label='Precision')
plt.plot(metrics['metrics/recall'], label='Recall')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Precision and Recall')
plt.legend()
plt.show()

### Plot mAP (mean Average Precision)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(metrics['metrics/mAP_0.5'], label='mAP@0.5')
plt.plot(metrics['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95')
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.title('Mean Average Precision')
plt.legend()
plt.show()

# Export the object detection model

In [ ]:
import os

save_dir = '/models/'

os.makedirs(save_dir, exist_ok=True)

export_path = '/models/license_plate_detector.onnx'

model.export(format='onnx', path= export_path)